In [22]:
import cv2
from keras.applications.resnet import ResNet50,preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
from keras.layers import Dense,Flatten,Dropout
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [23]:
def denoised_equalized(img):
    grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(np.uint8(grayimg))
    blurred = cv2.GaussianBlur(equalized_image, (1, 1), 0)
    denoised = cv2.medianBlur(blurred, 1)
    edges = cv2.Canny(denoised, 100, 200)
    contours, _ = cv2.findContours(
        edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(denoised)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    background_removed = cv2.bitwise_and(denoised, denoised, mask=mask)
    background_removed_rgb = cv2.cvtColor(
        background_removed, cv2.COLOR_BGR2RGB)
    return background_removed_rgb


In [24]:
height=300
width=300
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

In [25]:
train_dir="Dataset/Training"
validation_dir="Dataset/Validation"

batch_size=10

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)

validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(validation_dir,target_size=(height,width),batch_size=batch_size)

Found 35 images belonging to 2 classes.
Found 7 images belonging to 2 classes.


In [26]:
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False
    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)
    predictions=Dense(num_classes,activation='softmax')(x)
    finetune_model=Model(inputs=base_model.input,outputs=predictions) 
    return finetune_model

In [27]:
class_list = ['Real', 'Fake']
FC_Layers = [1024, 1024]
dropout = 0.5
num_epochs = 100
num_train_images = 35
metric = 'val_accuracy'


In [28]:
finetune_model = build_finetune_model(
    base_model, dropout=dropout, fc_layers=FC_Layers, num_classes=len(class_list))


In [29]:
checkpoint = ModelCheckpoint("Final_model.h5", monitor=metric, verbose=1,save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor=metric, min_delta=0,patience=40, verbose=1, mode="auto")
callbacks = [checkpoint, early]
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizers.Adam(lr=0.000001),metrics=['accuracy'])
finetune_model.fit_generator(generator=train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=callbacks)
finetune_model.save("Final_model.h5")


Epoch 1/100
3/3 [==============================] - ETA: 0s - loss: 2.2845 - accuracy: 0.2667
Epoch 1: val_accuracy improved from -inf to 0.71429, saving model to Final_model.h5
3/3 [==============================] - 31s 13s/step - loss: 2.2845 - accuracy: 0.2667 - val_loss: 0.7486 - val_accuracy: 0.7143
Epoch 2/100
3/3 [==============================] - ETA: 0s - loss: 1.3595 - accuracy: 0.6000
Epoch 2: val_accuracy did not improve from 0.71429
3/3 [==============================] - 4s 2s/step - loss: 1.3595 - accuracy: 0.6000 - val_loss: 1.1973 - val_accuracy: 0.4286
Epoch 3/100
3/3 [==============================] - ETA: 0s - loss: 1.0969 - accuracy: 0.6800
Epoch 3: val_accuracy improved from 0.71429 to 0.85714, saving model to Final_model.h5
3/3 [==============================] - 26s 12s/step - loss: 1.0969 - accuracy: 0.6800 - val_loss: 0.3654 - val_accuracy: 0.8571
Epoch 4/100
3/3 [==============================] - ETA: 0s - loss: 1.8925 - accuracy: 0.4000
Epoch 4: val_accuracy di

In [30]:
def equalize_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    equalized = equalized[:, :, np.newaxis]
    return equalized


In [40]:
from keras.utils import load_img
from keras.models import load_model


img=load_img("Dataset/Testing/Real.jpg",target_size=(300,300)) 
img=np.expand_dims(img,axis=0)
reconstructed_model = load_model("Final_model.h5")
output=reconstructed_model.predict(img) 
print(output)
if(output[0][0]>output[0][1]): print("Fake")
else:print("Real")


1/1 [==============================] - 1s 1s/step
[[0.07736683 0.9226331 ]]
Real


In [65]:
from PIL import ImageOps, ImageFilter


## Non Equalized Image

In [73]:
img = load_img("500-picture-cms.jpg", target_size=(300, 300))
img = np.expand_dims(img, axis=0)
output = reconstructed_model.predict(img)
print(output)
if(output[0][0] > output[0][1]):
    print("Fake")
else:
    print("Real")


1/1 [==============================] - 0s 212ms/step
[[0.45023856 0.5497614 ]]
Real


## Equalized Image

In [74]:
img = load_img("500-picture-cms.jpg", target_size=(300, 300))
img = ImageOps.equalize(img)
img = np.expand_dims(img, axis=0)
output = reconstructed_model.predict(img)
print(output)
if(output[0][0] > output[0][1]):
    print("Fake")
else:
    print("Real")


1/1 [==============================] - 0s 183ms/step
[[0.17750023 0.8224998 ]]
Real


## Median Blurred + Equalized

In [75]:
img = load_img("500-picture-cms.jpg", target_size=(300, 300))
img = ImageOps.equalize(img)
img = img.filter(ImageFilter.MedianFilter(size=3))
img = np.expand_dims(img, axis=0)
output = reconstructed_model.predict(img)
print(output)
if(output[0][0] > output[0][1]):
    print("Fake")
else:
    print("Real")


1/1 [==============================] - 0s 188ms/step
[[0.04372657 0.9562734 ]]
Real


# Teachable Machine Model

In [2]:
from keras.models import load_model  
from PIL import Image, ImageOps 
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)
model = load_model("keras_model.h5", compile=False)
class_names = open("labels.txt", "r").readlines()

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("g-1-380x214.jpg").convert("RGB")
# image = ImageOps.equalize(image)
# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 603ms/step
Class: Fake
Confidence Score: 0.6154035
